In [1]:
from math import nan
from future.utils import iteritems
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#from utils import get_data, generate_output, guess_human, seed_sequence, get_embeddings, find_closest
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
data = pd.read_csv("ner_dataset.csv", encoding= 'unicode_escape') 
data.head(672)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
667,NaN,Muslim,NNP,B-org
668,NaN,Brotherhood,NNP,I-org
669,NaN,as,IN,O
670,NaN,parts,NNS,O


In [3]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["Word"].values.tolist(),
                                                        s["Tag"].values.tolist())]
        self.grouped = self.dataset.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
          
getter = SentenceGetter(data)
sentences = getter.sentences

In [4]:
words = list(set(data["Word"].values))
nrofWords = len(words)

ner_tags = []
for tag in set(data["Tag"].values):
    if tag is nan or isinstance(tag, float):
        ner_tags.append('unk')
    else:
        ner_tags.append(tag)
nrofTags = len(ner_tags)
print(ner_tags)

['B-nat', 'B-tim', 'I-tim', 'B-per', 'I-art', 'I-nat', 'I-org', 'B-geo', 'I-geo', 'O', 'I-eve', 'I-per', 'B-gpe', 'I-gpe', 'B-eve', 'B-org', 'B-art']


In [5]:
index_word = {w: i for i, w in enumerate(words)}
index_tag = {t: i for i, t in enumerate(ner_tags)}
idx2tag = {v: k for k, v in iteritems(index_tag)}

In [6]:
maxlen = max([len(s) for s in sentences])
word_embedding_size = 17

X = [[index_word[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=nrofWords - 1)

y = [[index_tag[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=index_tag["O"])
y = [to_categorical(i, num_classes=nrofTags) for i in y]

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Simple RNN model

In [7]:
input = Input(shape=(maxlen,))

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=nrofWords+1, output_dim=32, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(32, return_sequences=True)),
    tf.keras.layers.Dense(nrofTags, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
])



In [8]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

model.summary()

history = model.fit(X_train, np.array(y_train), batch_size=32, epochs=30,
                    validation_split=0.5, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 32)             1125728   
_________________________________________________________________
bidirectional (Bidirectional (None, 1, 64)             4160      
_________________________________________________________________
dense (Dense)                (None, 1, 17)             1105      
_________________________________________________________________
dropout (Dropout)            (None, 1, 17)             0         
Total params: 1,130,993
Trainable params: 1,130,993
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
600/600 [==============================] - 5s 9ms/step - loss: 0.5991 - accuracy: 0.3131 - val_loss: 0.4533 - val_accuracy: 0.7135
Epoch 2/30
600/600 [==============================] - 5s 8ms/step - loss: 0.4649 - accuracy: 0.3636 - 

In [9]:
test_pred = model.predict(X_test, verbose=1)

idx2tag = {i: w for w, i in index_tag.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)

300/300 [==============================] - 0s 900us/step


In [10]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))
print(classification_report(test_labels, pred_labels))

F1-score: 72.6%
           precision    recall  f1-score   support

      tim       0.00      0.00      0.00       110
      geo       0.76      0.82      0.79       640
      per       0.78      0.78      0.78       819
      org       0.44      0.54      0.48       557
      gpe       0.93      0.89      0.91       591
      nat       0.00      0.00      0.00         3
      art       0.00      0.00      0.00         3
      eve       0.00      0.00      0.00         1

micro avg       0.72      0.73      0.73      2724
macro avg       0.70      0.73      0.72      2724



# LSTM model

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=nrofWords+1, output_dim=word_embedding_size, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Dense(nrofTags, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
])

In [12]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 17)             598043    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1, 128)            41984     
_________________________________________________________________
dense_1 (Dense)              (None, 1, 17)             2193      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 17)             0         
Total params: 642,220
Trainable params: 642,220
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit(X_train, np.array(y_train), batch_size=32, epochs=30,
                    validation_split=0.5, verbose=1)


Epoch 1/30
600/600 [==============================] - 4s 7ms/step - loss: 0.6221 - accuracy: 0.3548 - val_loss: 0.4864 - val_accuracy: 0.7135
Epoch 2/30
600/600 [==============================] - 3s 5ms/step - loss: 0.4765 - accuracy: 0.3639 - val_loss: 0.3849 - val_accuracy: 0.7135
Epoch 3/30
600/600 [==============================] - 3s 5ms/step - loss: 0.4498 - accuracy: 0.3658 - val_loss: 0.3636 - val_accuracy: 0.7135
Epoch 4/30
600/600 [==============================] - 3s 5ms/step - loss: 0.4425 - accuracy: 0.3648 - val_loss: 0.3552 - val_accuracy: 0.7135
Epoch 5/30
600/600 [==============================] - 3s 5ms/step - loss: 0.4389 - accuracy: 0.3597 - val_loss: 0.3507 - val_accuracy: 0.7135
Epoch 6/30
600/600 [==============================] - 4s 7ms/step - loss: 0.4381 - accuracy: 0.3606 - val_loss: 0.3475 - val_accuracy: 0.7135
Epoch 7/30
600/600 [==============================] - 4s 6ms/step - loss: 0.4348 - accuracy: 0.3571 - val_loss: 0.3447 - val_accuracy: 0.7135
Epoch 

In [15]:
test_pred = model.predict(X_test, verbose=1)

idx2tag = {i: w for w, i in index_tag.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_test)


300/300 [==============================] - 0s 1ms/step
[['O'], ['B-per'], ['O'], ['B-geo'], ['O'], ['B-per'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['B-geo'], ['B-per'], ['O'], ['O'], ['O'], ['O'], ['O'], ['B-gpe'], ['O'], ['O'], ['O'], ['O'], ['B-geo'], ['O'], ['O'], ['O'], ['O'], ['O'], ['B-geo'], ['O'], ['B-gpe'], ['O'], ['O'], ['O'], ['B-geo'], ['O'], ['O'], ['O'], ['B-gpe'], ['O'], ['O'], ['O'], ['B-per'], ['B-per'], ['O'], ['O'], ['O'], ['O'], ['B-per'], ['O'], ['B-geo'], ['O'], ['B-geo'], ['B-gpe'], ['O'], ['O'], ['B-geo'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['B-gpe'], ['O'], ['O'], ['O'], ['B-per'], ['O'], ['O'], ['O'], ['O'], ['B-per'], ['O'], ['O'], ['O'], ['B-geo'], ['O'], ['B-per'], ['O'], ['B-org'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['B-geo'], ['B-gpe'], ['O'], ['O'], ['O'], ['B-gpe'], ['B-gpe'], ['O'], ['O'], ['B-per'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['O'], ['B-per'], ['O'], ['O'], ['O'], ['O'], ['O']

In [16]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))

F1-score: 66.1%


In [17]:
print(classification_report(test_labels, pred_labels))

           precision    recall  f1-score   support

      tim       0.00      0.00      0.00       110
      geo       0.58      0.87      0.70       640
      per       0.66      0.82      0.73       819
      org       0.85      0.10      0.18       557
      gpe       0.74      0.89      0.81       591
      nat       0.00      0.00      0.00         3
      art       0.00      0.00      0.00         3
      eve       0.00      0.00      0.00         1

micro avg       0.66      0.67      0.66      2724
macro avg       0.67      0.67      0.60      2724

